In [ ]:
# import os
# import sys
# print(os.getcwd())
# current_dir = os.getcwd() 
# project_root = os.path.abspath(os.path.join(os.getcwd(), "..", "..", ".."))
# print(project_root)

# sys.path.append(project_root)



# from src.citibike.citibike_utils import get_trip_duration_mins
# from src.utils.datetime_utils import timestamp_to_date_col
# from pyspark.sql.functions import create_map, lit

d:\Udemy\DBA\pathfinder\dab_project\citibike_etl\notebooks\02_silver
d:\Udemy\DBA\pathfinder\dab_project


In [ ]:
from citibike.citibike_utils import get_trip_duration_mins
from utils.datetime_utils import timestamp_to_date_col
from pyspark.sql.functions import create_map, lit

In [ ]:
pipeline_id = dbutils.widgets.get("pipeline_id")
run_id = dbutils.widgets.get("run_id")
task_id = dbutils.widgets.get("task_id")
processed_timestamp = dbutils.widgets.get("processed_timestamp")
catalog = dbutils.widgets.get("catalog")



In [ ]:
df = spark.read.table(f"{catalog}.01_bronze.jc_citibike")
# df = spark.read.table(f"citibike_dev.01_bronze.jc_citibike")


In [5]:
df = get_trip_duration_mins(spark, df, "started_at", "ended_at", "trip_duration_mins")

In [8]:
df = timestamp_to_date_col(spark, df, "started_at", "trip_start_date")

In [ ]:
df = df.withColumn("metadata", 
              create_map(
                  lit("pipeline_id"), lit(pipeline_id),
                  lit("run_id"), lit(run_id),
                  lit("task_id"), lit(task_id),
                  lit("processed_timestamp"), lit(processed_timestamp)
                  ))

# df = df.withColumn("metadata", 
#               create_map(
#                   lit("pipeline_id"), lit("placeholder"),
#                   lit("run_id"), lit("placeholder"),
#                   lit("task_id"), lit("placeholder"),
#                   lit("processed_timestamp"), lit("placeholder")
#                   ))

In [11]:
df = df.select(
    "ride_id",
    "trip_start_date",
    "started_at",
    "ended_at",
    "start_station_name",
    "end_station_name",
    "trip_duration_mins",
    "metadata"
    )

In [ ]:
df.write.\
    mode("overwrite").\
    option("overwriteSchema", "true").\
    saveAsTable(f"{catalog}.02_silver.jc_citibike")

# df.write.\
#     mode("overwrite").\
#     option("overwriteSchema", "true").\
#     saveAsTable(f"citibike_dev.02_silver.jc_citibike")